In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.4 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModel

# Load Indic BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert",)
model = AutoModel.from_pretrained("ai4bharat/indic-bert", output_hidden_states=True)

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['sop_classifier.classifier.bias', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.weight', 'predictions.decoder.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
# Set device to CUDA if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

AlbertModel(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(200000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (attention_dropout): Dropout(p=0, inplace=False)
     

In [ ]:
# Define RNN model for sentiment analysis
class SentimentRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SentimentRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(device)
        out, hidden = self.rnn(x, h0)
        out = self.fc(hidden.squeeze(0))
        return out

In [ ]:
# Load labeled dataset of Hindi text for sentiment analysis
data = pd.read_csv("/content/drive/MyDrive/ABSA/input/sentiment_data.csv")


In [ ]:
# Tokenize input text using Indic BERT tokenizer
text_tokens = [tokenizer.encode(text, add_special_tokens=True) for text in data['text']]
input_ids = [torch.tensor(tokens).unsqueeze(0).to(device) for tokens in text_tokens]

In [ ]:
max_length = 128  # Maximum sequence length

with torch.no_grad():
    hidden_states = []
    for input_id in input_ids:
        truncated_input_id = input_id[:, :max_length] if input_id.size(1) > max_length else input_id  # Truncate input sequence if needed
        outputs = model(truncated_input_id)
        hidden_states.append(torch.cat(tuple(outputs[2][-4:]), dim=-1))
    features = torch.cat(hidden_states, dim=0)


In [ ]:
# Define input features for RNN model
input_size = features.shape[-1]
hidden_size = 256
output_size = 1



In [ ]:
# Instantiate RNN model
rnn_model = SentimentRNN(input_size, hidden_size, output_size).to(device)

In [ ]:
# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(rnn_model.parameters(), lr=0.001)

In [ ]:
# Convert labels to binary sentiment labels
labels = torch.tensor(data['experience'].values).float().unsqueeze(1).to(device)

In [ ]:
# Train RNN model
num_epochs = 10
batch_size = 32
num_batches = len(data) // batch_size
for epoch in range(num_epochs):
    for i in range(num_batches):
        # Get batch of input features and labels
        batch_features = features[i*batch_size:(i+1)*batch_size]
        batch_labels = labels[i*batch_size:(i+1)*batch_size]
        
        # Forward pass
        outputs = rnn_model(batch_features)
        loss = criterion(outputs, batch_labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print loss for every 10 batches
        if (i+1) % 10 == 0:
            print("Epoch [{}/{}], Batch [{}/{}], Loss: {:.4f}".format(epoch+1, num_epochs, i+1, num_batches, loss.item()))

Epoch [1/10], Batch [10/16], Loss: 0.6799
Epoch [2/10], Batch [10/16], Loss: 0.4821
Epoch [3/10], Batch [10/16], Loss: 0.2505
Epoch [4/10], Batch [10/16], Loss: 0.2459
Epoch [5/10], Batch [10/16], Loss: 0.2081
Epoch [6/10], Batch [10/16], Loss: 0.1861
Epoch [7/10], Batch [10/16], Loss: 0.5587
Epoch [8/10], Batch [10/16], Loss: 0.6298
Epoch [9/10], Batch [10/16], Loss: 0.0468
Epoch [10/10], Batch [10/16], Loss: 0.2510


In [ ]:
# Save trained RNN model weights
torch.save(rnn_model.state_dict(), "sentiment_rnn.pth")

In [ ]:
# Evaluate the model
rnn_model.eval()
with torch.no_grad():
    val_features = features[num_batches*batch_size:]  # Remaining features for validation
    val_labels = labels[num_batches*batch_size:]  # Remaining labels for validation
    val_outputs = rnn_model(val_features)
    val_loss = criterion(val_outputs, val_labels)
    print("Validation Loss: {:.4f}".format(val_loss.item()))

Validation Loss: 0.2401


In [ ]:
# Load trained RNN model weights
rnn_model = SentimentRNN(input_size, hidden_size, output_size).to(device)
rnn_model.load_state_dict(torch.load("sentiment_rnn.pth"))

# Tokenize new input text using Indic BERT tokenizer
new_text = "यह फिल्म खराब थी और इसे नहीं देखा जाना चाहिए।"
new_text_tokens = tokenizer.encode(new_text, add_special_tokens=True)
new_input_ids = torch.tensor(new_text_tokens).unsqueeze(0).to(device)

# Extract hidden states from Indic BERT model
with torch.no_grad():
    new_hidden_states = []
    new_outputs = model(new_input_ids)
    new_hidden_states.append(torch.cat(tuple(new_outputs[2][-4:]), dim=-1))
    new_features = torch.cat(new_hidden_states, dim=0)

# Pass hidden states through RNN model to predict sentiment
with torch.no_grad():
    new_rnn_output = rnn_model(new_features)
    new_prediction = torch.sigmoid(new_rnn_output).item()
    print("Sentiment prediction:", new_prediction)

Sentiment prediction: 0.0009380163974128664
